In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
def produce_brut(assureur):
    with open('/content/{}.json'.format(assureur), 'r',encoding="utf8") as f:
        data = json.load(f)
    pages_content=data["responses"]
    num_page=0
    df=[]
    for page in pages_content:
        num_page+=1
        if "fullTextAnnotation" not in page:
            continue
        p=page["fullTextAnnotation"]["pages"]
        for e in p:
            blocks=e["blocks"]
            page_features=[]
            for block in blocks:
                for para in block["paragraphs"]:
                    # collect text
                    text = ""
                    for word in para["words"]:
                        #print("-----")
                        #print(word)
                        for symbol in word["symbols"]:
                            if symbol["confidence"]>=0.8:
                                text += symbol["text"]
                        text+=" "
                    # extract bounding box features
                    x_list = []
                    y_list = []
                    for v in para["boundingBox"]["normalizedVertices"]:
                        x_list.append(v["x"])
                        y_list.append(v["y"])
                    f = {}
                    f["num_page"]=num_page
                    f["text"] = text
                    f["width"] = max(x_list) - min(x_list)
                    f["height"] = max(y_list) - min(y_list)
                    f["area"] = f["width"] * f["height"]
                    f["chars"] = len(text)
                    f["char_size"] = f["area"] / f["chars"] if f["chars"] > 0 else 0
                    f["pos_x"] = (f["width"] / 2.0) + min(x_list)
                    f["pos_y"] = (f["height"] / 2.0) + min(y_list)
                    f["aspect"] = f["width"] / f["height"] if f["height"] > 0 else 0
                    f["layout"] = "h" if f["aspect"] > 1 else "v"
                    f["x0"]=x_list[0]
                    f["x1"]=x_list[1]
                    f["y0"]=y_list[0]
                    f["y1"]=y_list[1]
                    page_features.append(f)
            df=df+page_features
    df=pd.DataFrame(df)
    df["assureur"]=assureur
    return df


# Exemple d'appel de la fonction et sauvegarde en CSV
assureur = "axa-output-1-to-71"  # Remplacez par le nom du fichier JSON sans l'extension
df_result = produce_brut(assureur)
df_result

,num_page,text,width,height,area,chars,char_size,pos_x,pos_y,aspect,layout,x0,x1,y0,y1,assureur
0,1,AA,0.104202,0.035672,0.003717,3,0.001239,0.142857,0.115339,2.921121,h,0.090756,0.194958,0.097503,0.098692,axa-output-1-to-71
1,1,AXA SA,0.247059,0.035672,0.008813,7,0.001259,0.568908,0.366231,6.925887,h,0.445378,0.692437,0.348395,0.348395,axa-output-1-to-71
2,1,Rapport,0.213445,0.034483,0.007360,8,0.000920,0.415966,0.469084,6.189918,h,0.309244,0.522689,0.451843,0.451843,axa-output-1-to-71
3,1,sur la Solvabilité,0.443698,0.036861,0.016355,19,0.000861,0.452101,0.522592,12.037077,h,0.230252,0.673950,0.507729,0.504162,axa-output-1-to-71
4,1,et la Situation Financière,0.667227,0.030916,0.020628,27,0.000764,0.479832,0.585018,21.582243,h,0.146219,0.813445,0.569560,0.569560,axa-output-1-to-71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1932,71,État S.23.01.02 Fonds propres,0.173109,0.010702,0.001853,30,0.000062,0.237815,0.532105,16.176016,h,0.151261,0.324370,0.526754,0.529132,axa-output-1-to-71
1933,71,État S.25.01.01 Capital de solvabilité requis ...,0.490756,0.009513,0.004668,88,0.000053,0.396639,0.554102,51.590674,h,0.151261,0.642017,0.549346,0.549346,axa-output-1-to-71
1934,71,État S.28.01.01 Minimum de capital requis act...,0.717647,0.024970,0.017920,162,0.000111,0.510084,0.582045,28.740038,h,0.151261,0.868908,0.569560,0.570749,axa-output-1-to-71
1935,71,71,0.016807,0.008323,0.000140,3,0.000047,0.497479,0.938763,2.019210,h,0.489076,0.505882,0.934602,0.934602,axa-output-1-to-71


Méthode 1 (Vision API)

In [30]:
import pandas as pd

def label_reports(data):
    """
    Fonction pour labéliser les blocs de texte issus de `produce_brut`.

    Args:
        data (pd.DataFrame): Tableau généré par `produce_brut()`.

    Returns:
        pd.DataFrame: Tableau enrichi avec une colonne "Label".
    """
    data.columns = data.columns.str.strip()  # Supprime les espaces inutiles
    def detect_title(text):
        """Détecter les titres basés sur des critères."""
        # Critères : texte court et majuscules
        if text.isupper() or len(text.split()) < 5:
            return True
        return False

    def detect_footer_or_header(text, position, total_blocks):
        """Détecter les bas/hauts de page."""
        # Bas ou haut de page basé sur la position et mots-clés
        if position < 3 or position > total_blocks - 3:  # Haut ou bas
            if any(keyword in text.lower() for keyword in ["page", "rapport", "confidentiel"]):
                return True
        return False

    def detect_paragraph(text):
        """Détecter les paragraphes."""
        # Critères : longueur du texte et structure
        if len(text.split()) > 10 and not text.isupper():
            return True
        return False
    print([col for col in data.columns])  # Affiche les colonnes avec des espaces visibles
    # Initialisation
    data["Label"] = "Inutile"  # Valeur par défaut
    total_blocks = len(data)  # Nombre total de blocs

    # Itérer sur les blocs
    for index, row in data.iterrows():
        text = row['text']  # Remplacer par le nom exact de la colonne contenant le texte
        position = index + 1

        # Vérification des critères
        if detect_footer_or_header(text, position, total_blocks):
            data.at[index, "Label"] = "Inutile"
        elif detect_title(text):
            data.at[index, "Label"] = "Titre"
        elif detect_paragraph(text):
            data.at[index, "Label"] = "Paragraphe"

    return data


Exemple d'exécution méthode 1

In [33]:
# Pipeline principal
if __name__ == "__main__":
    # Appeler produce_brut pour obtenir le DataFrame
    data = produce_brut("axa-output-1-to-71")  # Modifiez "nom_du_fichier_json" selon vos fichiers
    print("Extraction terminée. Tableaux sauvegardés.")
    print(label_reports(data))

Extraction terminée. Tableaux sauvegardés.
['num_page', 'text', 'width', 'height', 'area', 'chars', 'char_size', 'pos_x', 'pos_y', 'aspect', 'layout', 'x0', 'x1', 'y0', 'y1', 'assureur']
      num_page                                               text     width  \
0            1                                                AA   0.104202   
1            1                                            AXA SA   0.247059   
2            1                                           Rapport   0.213445   
3            1                                sur la Solvabilité   0.443698   
4            1                        et la Situation Financière   0.667227   
...        ...                                                ...       ...   
1932        71                     État S.23.01.02 Fonds propres   0.173109   
1933        71  État S.25.01.01 Capital de solvabilité requis ...  0.490756   
1934        71  État S.28.01.01 Minimum de capital requis  act...  0.717647   
1935        71         

Méthode 2 (méthode hybride)

In [37]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.preprocessing import LabelEncoder
import re

# Charger un modèle pré-entraîné Hugging Face
MODEL_NAME = "distilbert-base-uncased"  # Modèle NLP pré-entraîné
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3)

# Classes pour la classification
LABELS = ["Inutile", "Titre", "Paragraphe"]
label_encoder = LabelEncoder()
label_encoder.fit(LABELS)

# Fonction pour classifier les lignes
def classify_lines(lines):
    """
    Classifie chaque ligne en "Inutile", "Titre" ou "Paragraphe" à l'aide de BERT.

    Args:
        lines (list of str): Liste des lignes de texte.

    Returns:
        list of str: Liste des labels correspondants.
    """
    results = []
    for line in lines:
        inputs = tokenizer(line, return_tensors="pt", truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            predicted_class = torch.argmax(logits, dim=1).item()
            results.append(label_encoder.inverse_transform([predicted_class])[0])
    return results

# Détecter les tableaux dans le texte
def detect_tables(input_text):
    """
    Identifie les tableaux en repérant des motifs d'espacement dans le texte.

    Args:
        input_text (str): Texte brut.

    Returns:
        list of str: Liste des blocs correspondant à des tableaux.
    """
    table_pattern = r'\s{2,}'  # Deux espaces ou plus
    return [block for block in input_text.split('\n\n') if re.search(table_pattern, block)]

# Traiter les données brutes du DataFrame retourné par produce_brut()
def process_dataframe(df):
    """
    Traite un DataFrame produit par produce_brut() : classifie les lignes et extrait les tableaux.

    Args:
        df (pd.DataFrame): DataFrame contenant les blocs textuels avec métadonnées.

    Returns:
        pd.DataFrame: DataFrame contenant les lignes classifiées.
        list of pd.DataFrame: Liste des tableaux extraits sous forme de DataFrame.
    """
    # Extraire le texte des blocs
    lines = df['text'].tolist()

    # Classifier les lignes
    print("Classification des lignes en cours...")
    line_labels = classify_lines(lines)

    # Ajouter les labels au DataFrame
    df['Label'] = line_labels

    # Identifier les tableaux
    print("Extraction des tableaux en cours...")
    tables_text = detect_tables("\n".join(lines))
    tables = [pd.DataFrame([re.split(r'\s{2,}', row) for row in table.split('\n')]) for table in tables_text]

    return df, tables

# Fonction principale
if __name__ == "__main__":
    # Simuler le DataFrame retourné par produce_brut()
    data = produce_brut("axa-output-1-to-71")
    df = pd.DataFrame(data)

    # Traiter les données
    classified_df, tables = process_dataframe(df)

    # Afficher les résultats
    print("\nContenu Classifié :")
    print(classified_df)

    print("\nTableaux Extraits :")
    for i, table in enumerate(tables):
        print(f"Tableau {i + 1}:")
        print(table)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Classification des lignes en cours...
Extraction des tableaux en cours...

Contenu Classifié :
      num_page                                               text     width  \
0            1                                                AA   0.104202   
1            1                                            AXA SA   0.247059   
2            1                                           Rapport   0.213445   
3            1                                sur la Solvabilité   0.443698   
4            1                        et la Situation Financière   0.667227   
...        ...                                                ...       ...   
1932        71                     État S.23.01.02 Fonds propres   0.173109   
1933        71  État S.25.01.01 Capital de solvabilité requis ...  0.490756   
1934        71  État S.28.01.01 Minimum de capital requis  act...  0.717647   
1935        71                                                71   0.016807   
1936        71                      

On voit que le modèle utilisé a besoin d'entainement sur des bases de données qui contient des colonnes 'text' et 'label'. On a donc pris les résultats générés de la première méthode.

In [55]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [52]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
data['encoded_label'] = label_encoder.fit_transform(data['Label'])

# Afficher le mapping des labels
print("Mapping des labels :", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))

Mapping des labels : {'Inutile': 0, 'Paragraphe': 1, 'Titre': 2}


In [56]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Charger le tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Diviser les données en train/test
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['text'].tolist(),
    data['encoded_label'].tolist(),
    test_size=0.2,  # 20% pour validation
    random_state=42
)

# Tokenizer les données
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

# Créer des datasets pour Hugging Face
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"],
                                   "attention_mask": train_encodings["attention_mask"],
                                   "labels": train_labels})

val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"],
                                 "attention_mask": val_encodings["attention_mask"],
                                 "labels": val_labels})


In [58]:
# Charger le modèle avec le bon nombre de labels
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)
import os
os.environ["WANDB_DISABLED"] = "true"
# Définir les arguments d'entraînement
training_args = TrainingArguments(
    output_dir="./results",          # Répertoire pour sauvegarder les modèles
    eval_strategy="epoch",    # Évaluer à chaque epoch
    learning_rate=2e-5,             # Taux d'apprentissage
    per_device_train_batch_size=16, # Batch size pour entraînement
    per_device_eval_batch_size=16,  # Batch size pour validation
    num_train_epochs=3,             # Nombre d'epochs
    weight_decay=0.01,              # Regularisation L2
    logging_dir="./logs",           # Répertoire pour logs
    logging_steps=10                # Logs tous les 10 pas
)

# Configurer le Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Entraîner le modèle
trainer.train()

# Sauvegarder le modèle fine-tuné
model.save_pretrained("./fine_tuned_distilbert")
tokenizer.save_pretrained("./fine_tuned_distilbert")
print("Modèle fine-tuné sauvegardé !")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss
1,0.158000,0.145120
2,0.051100,0.073635
3,0.022300,0.104147


Modèle fine-tuné sauvegardé !
